In [1]:
import numpy, pandas
import os

In [2]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
import random
from functools import reduce

import json, os
import traceback
import matplotlib.pyplot as plt
%matplotlib inline

import wget

In [3]:
from groupify.app.spotify_utils import *
from groupify.app.ml_utils import *

In [4]:
redirect_uri = 'https://example.com'

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import groupify.app.psecrets as psecrets

try: 
    cid = psecrets.client_id
    secret = psecrets.secret
    userdata = json.load(open('groupify/app/json/userdata.json', 'r'))
    username = list(userdata.keys())[0]
    scope = 'user-top-read user-read-recently-played user-follow-read playlist-modify-public user-library-read playlist-read-collaborative'
    token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
except: 
    print(traceback.format_exc(5))


In [6]:
list_of_genres = pandas.read_html('https://everynoise.com/everynoise1d.cgi?scope=all', index_col=0)
list_of_genres = list_of_genres[0].replace('☊', '')
list_of_genres = list_of_genres.values.tolist()
list_of_genres = [a[1] for a in list_of_genres]
list_of_genres

['pop',
 'dance pop',
 'rap',
 'rock',
 'hip hop',
 'latin',
 'pop rap',
 'trap latino',
 'trap',
 'modern rock',
 'edm',
 'reggaeton',
 'pop dance',
 'post-teen pop',
 'latin pop',
 'electropop',
 'pop rock',
 'classic rock',
 'tropical house',
 'r&b',
 'mellow gold',
 'musica mexicana',
 'soft rock',
 'permanent wave',
 'album rock',
 'melodic rap',
 'southern hip hop',
 'alternative metal',
 'k-pop',
 'urban contemporary',
 'canadian pop',
 'uk pop',
 'electro house',
 'contemporary country',
 'indie pop',
 'alternative rock',
 'alt z',
 'gangster rap',
 'nueva musica mexicana',
 'hard rock',
 'nu metal',
 'country',
 'ranchera',
 'rock en espanol',
 'tropical',
 'latin hip hop',
 'desi pop',
 'art pop',
 'neo mellow',
 'underground hip hop',
 'adult standards',
 'atl hip hop',
 'modern alternative rock',
 'french hip hop',
 'country road',
 'folk rock',
 'arrocha',
 'chicago rap',
 'filmi',
 'new wave pop',
 'pop urbaine',
 'german hip hop',
 'art rock',
 'soul',
 'country rock',
 

In [7]:
num_playlists=3
num_genres = 10
genres_per_playlist = 3

simulated_playlists = []

for ind in range(num_playlists): 
    pl_indexes = numpy.random.choice(num_genres, genres_per_playlist, replace=False) # 3 genres per playlist
    genres = [list_of_genres[a] for a in pl_indexes]
    print('Generating playlists for these genres: ', genres)
    trackids = get_recommendations_tracks_from_genre(sp, genres, limit=10)
    print(len(trackids))
    simulated_playlists.append(trackids)
# print([len(a) for a in simulated_playlists])

Generating playlists for these genres:  ['modern rock', 'trap', 'hip hop']
0
Generating playlists for these genres:  ['trap latino', 'latin', 'dance pop']
109
Generating playlists for these genres:  ['latin', 'pop', 'dance pop']
103


## Goal: 
Given a playlist parameterized by list of track_ids, we want to 
1) generate a feature matrix of dimension num_tracks_in_playlist x num_features
2) Perform unsupervised learning to find a latent representation that maximizes the dimensions that are common to both playlists --- baseline PCA, ideal: autoencoders. 

## Function definitions for features extraction 

In [8]:
def get_genre_from_track(sp, trackid, limit=1): 
    track = sp.track(trackid)
    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    genres = artist["genres"]
    if len(genres): 
        return genres[0:limit]
    else: 
        return []

In [9]:
def get_audio_df(sp, trackids, numerical_features, categorical_features): 

    feature_df = pandas.DataFrame(sp.audio_features(trackids))
    feature_df = feature_df[numerical_features + categorical_features ]
    feature_df[categorical_features] = feature_df[categorical_features].astype(str)
    feature_df = pandas.get_dummies(feature_df, drop_first = True, columns = categorical_features)
    feature_df['id'] = trackids
    return feature_df

In [10]:
def get_stats(arr, col_prefix='', pvalues = [0, 25, 50, 75,100]): 
    assert isinstance(arr, numpy.ndarray) or isinstance(arr, list)
    if isinstance(arr, list): 
        arr = numpy.array(arr)
    percentiles = numpy.round(numpy.percentile(arr, pvalues, axis = 0), decimals = 2).tolist()
    pvalues = [col_prefix + '_' + str(p) for p in pvalues]
    d = dict(zip(pvalues, percentiles))
    return d

In [11]:
get_stats(numpy.random.rand(5), 'test')

{'test_0': 0.08,
 'test_25': 0.22,
 'test_50': 0.5,
 'test_75': 0.87,
 'test_100': 0.92}

In [12]:
def get_mean_variance(arr, col_prefix=''): 
    d = {}
    d[col_prefix + '_mean'] = numpy.round(numpy.mean(arr, axis = None), 2)
    d[col_prefix + '_std'] = numpy.round(numpy.std(arr, axis = None), 2)
    return d

In [13]:
from collections import Counter
import librosa

def get_extensive_audio_features(sp, sample_trackid, pvalues= [10,25,50,75,90], perform_dsp = True): 
    '''
    Get a single track and retrieve audio features that are in addition to the basic features.
    '''
    pitches = 'C C# D D# E F F# G G# A A# B'.split(' ')
    timbres = ['timbre_'+str(a) for a in range(12)]

    sample_audio_analysis = sp.audio_analysis(sample_trackid)

    # processing a single track 
    num_sections = len(sample_audio_analysis['sections'])
    features = {'id': sample_trackid}
    loudness = [sample_audio_analysis['sections'][a]['loudness'] for a in range(num_sections)]
    tempo = [sample_audio_analysis['sections'][a]['tempo'] for a in range(num_sections)]
    features.update(get_stats(numpy.array(loudness), 'loudness'))
    features.update(get_stats(numpy.array(tempo), 'tempo'))

    key = ['key_'+str(sample_audio_analysis['sections'][a]['key']) for a in range(num_sections)]
    mode = ['mode_'+str(sample_audio_analysis['sections'][a]['mode']) for a in range(num_sections)]
    key_counter = dict(Counter(key))
    _sum = sum(key_counter.values())
    key_counter = {a:round(b/_sum*100)/100. for a,b in key_counter.items()}
    mode_counter = dict(Counter(mode))
    _sum = sum(mode_counter.values())
    mode_counter = {a:round(b/_sum*100)/100. for a,b in mode_counter.items()}

    features.update(key_counter)
    features.update(mode_counter)


    # pitches and timbre
    num_segments = len(sample_audio_analysis['segments'])
    pitch_data = []
    timbre_data = []
    for a in range(num_segments): 
        pitch_data.append(sample_audio_analysis['segments'][a]['pitches'])
        timbre_data.append(sample_audio_analysis['segments'][a]['timbre'])
    pitch_df = pandas.DataFrame(data = numpy.array(pitch_data), columns = pitches)
    timbre_df = pandas.DataFrame(data = numpy.array(timbre_data), columns = timbres)

    for p in pitches: 
        features.update(get_stats(pitch_df[p].values, p, pvalues = [10,25,50,75,90]))
    for t in timbres: 
        features.update(get_stats(timbre_df[t].values, t, pvalues = [10,25,50,75,90]))
        
    if perform_dsp: 
        sample_track = sp.track(sample_trackid)
        audio_file = wget.download(sample_track['preview_url'])
        audio, sr  = librosa.load(audio_file)
        os.remove(audio_file)

        zero_crossings = librosa.feature.zero_crossing_rate(audio)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)

        features.update(get_mean_variance(zero_crossings, col_prefix='zero_crossings'))
        features.update(get_mean_variance(spectral_centroids, col_prefix='spectral_centroids'))
        features.update(get_mean_variance(rolloff, col_prefix='rolloff'))
        
    return features
    


## Testing Features on a single track 

In [14]:
trackids = get_recommendations_tracks_from_genre(sp,['classical'], limit=5)
sample_trackid = trackids[5]

In [16]:
# sp.audio_features(trackids)[5]

In [17]:
numerical_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness']
numerical_features += ['valence', 'tempo', 'duration_ms']
categorical_features = ['key', 'mode']
feature_df = get_audio_df(sp, trackids, numerical_features, categorical_features)
feature_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key_11,key_2,key_4,key_5,key_6,key_7,key_8,key_9,mode_1,id
0,0.304,0.6570,-13.567,0.0824,0.787,0.954,0.113,0.1020,120.338,162133,...,0,0,0,0,0,0,0,0,1,1JSIWsJfxOji0FrxFcxdCK
1,0.295,0.0847,-25.278,0.0297,0.979,0.962,0.114,0.2790,47.056,75693,...,0,0,1,0,0,0,0,0,0,5QnJqzHud1dXiz0nohQtbv
2,0.266,0.2130,-15.952,0.0383,0.991,0.878,0.110,0.0323,99.451,101867,...,0,0,0,0,0,0,0,1,1,7LnCjHudaBcB7gprluL2jG
3,0.326,0.5200,-10.204,0.0478,0.930,0.495,0.352,0.1320,128.029,67067,...,0,0,0,0,0,0,0,0,1,1S1JlOrGDpkbVhWCsOjUIh
4,0.298,0.2170,-15.660,0.0321,0.941,0.388,0.109,0.1470,90.713,111933,...,0,0,0,0,0,1,0,0,1,1J8JRBcaJUPctNt6DM12iD


In [18]:
# extensive features for sample track 

get_extensive_audio_features(sp, sample_trackid, pvalues= [10,25,50,75,90], perform_dsp = True)


100% [............................................................................] 362861 / 362861

/Users/tejasbot/anaconda3/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:56: FutureWarning: Pass y=[-0.01572599 -0.03161804 -0.04182438 ... -0.0389565  -0.05666606
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectral_centroids = librosa.feature.spectral_centroid(audio, sr=sr)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:57: FutureWarning: Pass y=[-0.01572599 -0.03161804 -0.04182438 ... -0.0389565  -0.05666606
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = librosa.feature.spectral_rolloff(audio, sr=sr)


{'id': '69ai605TdgykOy3UTKuywI',
 'loudness_0': -33.95,
 'loudness_25': -27.29,
 'loudness_50': -23.17,
 'loudness_75': -19.82,
 'loudness_100': -18.24,
 'tempo_0': 0.0,
 'tempo_25': 67.74,
 'tempo_50': 81.68,
 'tempo_75': 82.87,
 'tempo_100': 84.57,
 'key_8': 0.12,
 'key_10': 0.5,
 'key_3': 0.25,
 'key_7': 0.12,
 'mode_1': 0.62,
 'mode_0': 0.38,
 'C_10': 0.12,
 'C_25': 0.22,
 'C_50': 0.43,
 'C_75': 0.7,
 'C_90': 1.0,
 'C#_10': 0.11,
 'C#_25': 0.19,
 'C#_50': 0.34,
 'C#_75': 0.59,
 'C#_90': 0.94,
 'D_10': 0.1,
 'D_25': 0.21,
 'D_50': 0.41,
 'D_75': 0.73,
 'D_90': 1.0,
 'D#_10': 0.02,
 'D#_25': 0.06,
 'D#_50': 0.25,
 'D#_75': 0.59,
 'D#_90': 0.98,
 'E_10': 0.03,
 'E_25': 0.07,
 'E_50': 0.14,
 'E_75': 0.24,
 'E_90': 0.43,
 'F_10': 0.08,
 'F_25': 0.16,
 'F_50': 0.33,
 'F_75': 0.62,
 'F_90': 1.0,
 'F#_10': 0.03,
 'F#_25': 0.06,
 'F#_50': 0.12,
 'F#_75': 0.26,
 'F#_90': 0.41,
 'G_10': 0.06,
 'G_25': 0.14,
 'G_50': 0.34,
 'G_75': 0.66,
 'G_90': 1.0,
 'G#_10': 0.03,
 'G#_25': 0.05,
 'G#_50': 

In [19]:
# create a dataframe with these extensive features

features_all_tracks = []
for t_id in trackids: 
    print('processing track ', t_id)
    features_all_tracks.append(get_extensive_audio_features(sp, t_id, pvalues= [10,25,50,75,90], perform_dsp = True))

df_audio_extra = pandas.DataFrame(features_all_tracks)



processing track  1JSIWsJfxOji0FrxFcxdCK
100% [............................................................................] 362861 / 362861

/Users/tejasbot/anaconda3/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:56: FutureWarning: Pass y=[-0.02529239 -0.01567331  0.01690556 ... -0.03486066 -0.11116249
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectral_centroids = librosa.feature.spectral_centroid(audio, sr=sr)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:57: FutureWarning: Pass y=[-0.02529239 -0.01567331  0.01690556 ... -0.03486066 -0.11116249
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = librosa.feature.spectral_rolloff(audio, sr=sr)


processing track  5QnJqzHud1dXiz0nohQtbv
100% [............................................................................] 361837 / 361837

/Users/tejasbot/anaconda3/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:56: FutureWarning: Pass y=[-0.0111036  -0.01993817 -0.01985832 ... -0.02482001 -0.02549613
 -0.02737209] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectral_centroids = librosa.feature.spectral_centroid(audio, sr=sr)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:57: FutureWarning: Pass y=[-0.0111036  -0.01993817 -0.01985832 ... -0.02482001 -0.02549613
 -0.02737209] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = librosa.feature.spectral_rolloff(audio, sr=sr)


processing track  7LnCjHudaBcB7gprluL2jG
100% [............................................................................] 362861 / 362861

...

/Users/tejasbot/anaconda3/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:56: FutureWarning: Pass y=[-0.04906115 -0.10394172 -0.12693784 ... -0.03218609 -0.01424022
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectral_centroids = librosa.feature.spectral_centroid(audio, sr=sr)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:57: FutureWarning: Pass y=[-0.04906115 -0.10394172 -0.12693784 ... -0.03218609 -0.01424022
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = librosa.feature.spectral_rolloff(audio, sr=sr)


processing track  2ddhCHZW8CT0ZVJG2lHyuM
100% [............................................................................] 362861 / 362861

/Users/tejasbot/anaconda3/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:56: FutureWarning: Pass y=[0.01085229 0.02702581 0.03758576 ... 0.12922898 0.11204145 0.0869234 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectral_centroids = librosa.feature.spectral_centroid(audio, sr=sr)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:57: FutureWarning: Pass y=[0.01085229 0.02702581 0.03758576 ... 0.12922898 0.11204145 0.0869234 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = librosa.feature.spectral_rolloff(audio, sr=sr)


processing track  1DeWXeOfjt3Hj3usUQfMYN
100% [............................................................................] 362861 / 362861

/Users/tejasbot/anaconda3/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:56: FutureWarning: Pass y=[0.02524064 0.04758297 0.05699204 ... 0.09006309 0.0926076  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectral_centroids = librosa.feature.spectral_centroid(audio, sr=sr)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_48346/3714470853.py:57: FutureWarning: Pass y=[0.02524064 0.04758297 0.05699204 ... 0.09006309 0.0926076  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = librosa.feature.spectral_rolloff(audio, sr=sr)


In [20]:
df_audio_extra.head(10)

,id,loudness_0,loudness_25,loudness_50,loudness_75,loudness_100,tempo_0,tempo_25,tempo_50,tempo_75,...,key_10,key_4,key_9,key_8,key_3,key_11,key_6,key_1,key_-1,mode_-1
0,1JSIWsJfxOji0FrxFcxdCK,-37.15,-24.38,-16.22,-11.01,-9.65,100.14,108.83,118.57,122.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5QnJqzHud1dXiz0nohQtbv,-28.85,-28.55,-28.26,-24.71,-21.16,45.41,46.40,47.39,47.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7LnCjHudaBcB7gprluL2jG,-33.94,-23.38,-18.88,-15.86,-10.38,98.10,98.97,100.37,101.27,...,0.17,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1S1JlOrGDpkbVhWCsOjUIh,-15.84,-14.33,-12.82,-10.10,-7.37,126.25,127.29,128.34,129.05,...,NaN,NaN,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1J8JRBcaJUPctNt6DM12iD,-22.08,-20.58,-18.01,-15.02,-12.28,88.55,90.99,92.15,92.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,69ai605TdgykOy3UTKuywI,-33.95,-27.29,-23.17,-19.82,-18.24,0.00,67.74,81.68,82.87,...,0.50,NaN,NaN,0.12,0.25,NaN,NaN,NaN,NaN,NaN
6,711sUHL5feGZQ4HZai697P,-23.46,-20.55,-16.85,-13.01,-10.89,108.52,109.38,111.54,116.88,...,0.14,NaN,0.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5pLpGimrruc4qZSxHBScYz,-47.09,-37.80,-19.52,-16.91,-10.27,58.59,82.40,85.58,86.91,...,0.06,NaN,0.18,0.06,NaN,0.06,NaN,NaN,NaN,NaN
8,7igDc2V0gCwBYV5rBUw1g7,-30.93,-27.92,-21.31,-16.93,-15.60,69.14,74.56,100.69,101.27,...,0.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,777Em4kHDduR6HgVTKdwpf,-35.02,-28.04,-23.44,-20.81,-18.30,0.00,59.72,62.42,64.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_audio_extra.columns.tolist()
print(df_audio_extra.dtypes)
print(feature_df.dtypes)

id              object
loudness_0     float64
loudness_25    float64
loudness_50    float64
loudness_75    float64
                ...   
key_11         float64
key_6          float64
key_1          float64
key_-1         float64
mode_-1        float64
Length: 153, dtype: object
danceability        float64
energy              float64
loudness            float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms           int64
key_1                 uint8
key_10                uint8
key_11                uint8
key_2                 uint8
key_4                 uint8
key_5                 uint8
key_6                 uint8
key_7                 uint8
key_8                 uint8
key_9                 uint8
mode_1                uint8
id                   object
dtype: object


In [29]:
## join the basic df with the extensive_audio df

joined_df = pandas.merge(feature_df, df_audio_extra, how='inner', on='id')
print(joined_df.columns.tolist())
joined_df.head()


['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'key_1_x', 'key_10_x', 'key_11_x', 'key_2_x', 'key_4_x', 'key_5_x', 'key_6_x', 'key_7_x', 'key_8_x', 'key_9_x', 'mode_1_x', 'id', 'loudness_0', 'loudness_25', 'loudness_50', 'loudness_75', 'loudness_100', 'tempo_0', 'tempo_25', 'tempo_50', 'tempo_75', 'tempo_100', 'key_0', 'key_2_y', 'mode_1_y', 'mode_0', 'C_10', 'C_25', 'C_50', 'C_75', 'C_90', 'C#_10', 'C#_25', 'C#_50', 'C#_75', 'C#_90', 'D_10', 'D_25', 'D_50', 'D_75', 'D_90', 'D#_10', 'D#_25', 'D#_50', 'D#_75', 'D#_90', 'E_10', 'E_25', 'E_50', 'E_75', 'E_90', 'F_10', 'F_25', 'F_50', 'F_75', 'F_90', 'F#_10', 'F#_25', 'F#_50', 'F#_75', 'F#_90', 'G_10', 'G_25', 'G_50', 'G_75', 'G_90', 'G#_10', 'G#_25', 'G#_50', 'G#_75', 'G#_90', 'A_10', 'A_25', 'A_50', 'A_75', 'A_90', 'A#_10', 'A#_25', 'A#_50', 'A#_75', 'A#_90', 'B_10', 'B_25', 'B_50', 'B_75', 'B_90', 'timbre_0_10', 'timbre_0_25', 'timbre_0_50', 'timbr

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key_10_y,key_4_y,key_9_y,key_8_y,key_3,key_11_y,key_6_y,key_1_y,key_-1,mode_-1
0,0.304,0.6570,-13.567,0.0824,0.787,0.954,0.113,0.1020,120.338,162133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.295,0.0847,-25.278,0.0297,0.979,0.962,0.114,0.2790,47.056,75693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.266,0.2130,-15.952,0.0383,0.991,0.878,0.110,0.0323,99.451,101867,...,0.17,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.326,0.5200,-10.204,0.0478,0.930,0.495,0.352,0.1320,128.029,67067,...,NaN,NaN,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.298,0.2170,-15.660,0.0321,0.941,0.388,0.109,0.1470,90.713,111933,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
## sanity 

print(len(feature_df), len(df_audio_extra), len(joined_df), len(trackids))

100 100 100 100


In [109]:
# figure out loudness, timbre, tempo pitch and add it to main feature_df -- 1 hr
from collections import Counter
import librosa

pitches = 'C C# D D# E F F# G G# A A# B'.split(' ')
timbres = ['timbre_'+str(a) for a in range(12)]

sample_audio_analysis = sp.audio_analysis(sample_trackid)

# processing a single track 
num_sections = len(sample_audio_analysis['sections'])
features = {'id': sample_trackid}
loudness = [sample_audio_analysis['sections'][a]['loudness'] for a in range(num_sections)]
tempo = [sample_audio_analysis['sections'][a]['tempo'] for a in range(num_sections)]
features.update(get_stats(numpy.array(loudness), 'loudness'))
features.update(get_stats(numpy.array(tempo), 'tempo'))

key = ['key_'+str(sample_audio_analysis['sections'][a]['key']) for a in range(num_sections)]
mode = ['mode_'+str(sample_audio_analysis['sections'][a]['mode']) for a in range(num_sections)]
key_counter = dict(Counter(key))
_sum = sum(key_counter.values())
key_counter = {a:round(b/_sum*100)/100. for a,b in key_counter.items()}
mode_counter = dict(Counter(mode))
_sum = sum(mode_counter.values())
mode_counter = {a:round(b/_sum*100)/100. for a,b in mode_counter.items()}

features.update(key_counter)
features.update(mode_counter)


# pitches and timbre
num_segments = len(sample_audio_analysis['segments'])
pitch_data = []
timbre_data = []
for a in range(num_segments): 
    pitch_data.append(sample_audio_analysis['segments'][a]['pitches'])
    timbre_data.append(sample_audio_analysis['segments'][a]['timbre'])
pitch_df = pandas.DataFrame(data = numpy.array(pitch_data), columns = pitches)
timbre_df = pandas.DataFrame(data = numpy.array(timbre_data), columns = timbres)

for p in pitches: 
    features.update(get_stats(pitch_df[p].values, p, pvalues = [10,25,50,75,90]))
for t in timbres: 
    features.update(get_stats(timbre_df[t].values, t, pvalues = [10,25,50,75,90]))

print(features)
 


{'loudness_0': -41.22, 'loudness_25': -38.55, 'loudness_50': -30.35, 'loudness_75': -25.81, 'loudness_100': -20.09, 'tempo_0': 129.81, 'tempo_25': 135.34, 'tempo_50': 136.87, 'tempo_75': 140.0, 'tempo_100': 146.34, 'key_11': 0.07, 'key_1': 0.33, 'key_6': 0.2, 'key_0': 0.07, 'key_8': 0.2, 'key_4': 0.07, 'key_5': 0.07, 'mode_1': 0.8, 'mode_0': 0.2, 'C_10': 0.04, 'C_25': 0.07, 'C_50': 0.14, 'C_75': 0.31, 'C_90': 0.89, 'C#_10': 0.04, 'C#_25': 0.07, 'C#_50': 0.15, 'C#_75': 0.38, 'C#_90': 0.94, 'D_10': 0.03, 'D_25': 0.05, 'D_50': 0.11, 'D_75': 0.23, 'D_90': 0.64, 'D#_10': 0.05, 'D#_25': 0.1, 'D#_50': 0.24, 'D#_75': 0.81, 'D#_90': 1.0, 'E_10': 0.04, 'E_25': 0.06, 'E_50': 0.12, 'E_75': 0.25, 'E_90': 0.73, 'F_10': 0.03, 'F_25': 0.06, 'F_50': 0.14, 'F_75': 0.47, 'F_90': 1.0, 'F#_10': 0.02, 'F#_25': 0.04, 'F#_50': 0.1, 'F#_75': 0.22, 'F#_90': 0.65, 'G_10': 0.02, 'G_25': 0.04, 'G_50': 0.09, 'G_75': 0.17, 'G_90': 0.38, 'G#_10': 0.04, 'G#_25': 0.08, 'G#_50': 0.16, 'G#_75': 0.44, 'G#_90': 0.96, 'A_10

In [114]:
sample_track = sp.track(sample_trackid)
audio_file = wget.download(sample_track['preview_url'])
audio, sr  = librosa.load(audio_file)
os.remove(audio_file)

zero_crossings = librosa.feature.zero_crossing_rate(audio)
spectral_centroids = librosa.feature.spectral_centroid(audio, sr=sr)
rolloff = librosa.feature.spectral_rolloff(audio, sr=sr)

features.update(get_mean_variance(zero_crossings, col_prefix='zero_crossings'))
features.update(get_mean_variance(spectral_centroids, col_prefix='spectral_centroids'))
features.update(get_mean_variance(rolloff, col_prefix='rolloff'))


#features.update({'zero_crossings_mean': numpy.mean(zero_crossings, axis = None), 'zero_crossings_std': numpy.std(numpy.std(zero_crossings, axis = None))})
#features.update({'spectral_centroids_mean': numpy.mean(spectral_centroids, axis = None), 'spectral_centroids_std': numpy.std(numpy.std(spectral_centroids, axis = None))})
#features.update({'rolloff_mean': numpy.mean(rolloff, axis = None), 'rolloff_std': numpy.std(numpy.std(rolloff, axis = None))})

print(features)

100% [............................................................................] 362861 / 362861

/Users/tejasbot/anaconda3/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


{'loudness_0': -41.22, 'loudness_25': -38.55, 'loudness_50': -30.35, 'loudness_75': -25.81, 'loudness_100': -20.09, 'tempo_0': 129.81, 'tempo_25': 135.34, 'tempo_50': 136.87, 'tempo_75': 140.0, 'tempo_100': 146.34, 'key_11': 0.07, 'key_1': 0.33, 'key_6': 0.2, 'key_0': 0.07, 'key_8': 0.2, 'key_4': 0.07, 'key_5': 0.07, 'mode_1': 0.8, 'mode_0': 0.2, 'C_10': 0.04, 'C_25': 0.07, 'C_50': 0.14, 'C_75': 0.31, 'C_90': 0.89, 'C#_10': 0.04, 'C#_25': 0.07, 'C#_50': 0.15, 'C#_75': 0.38, 'C#_90': 0.94, 'D_10': 0.03, 'D_25': 0.05, 'D_50': 0.11, 'D_75': 0.23, 'D_90': 0.64, 'D#_10': 0.05, 'D#_25': 0.1, 'D#_50': 0.24, 'D#_75': 0.81, 'D#_90': 1.0, 'E_10': 0.04, 'E_25': 0.06, 'E_50': 0.12, 'E_75': 0.25, 'E_90': 0.73, 'F_10': 0.03, 'F_25': 0.06, 'F_50': 0.14, 'F_75': 0.47, 'F_90': 1.0, 'F#_10': 0.02, 'F#_25': 0.04, 'F#_50': 0.1, 'F#_75': 0.22, 'F#_90': 0.65, 'G_10': 0.02, 'G_25': 0.04, 'G_50': 0.09, 'G_75': 0.17, 'G_90': 0.38, 'G#_10': 0.04, 'G#_25': 0.08, 'G#_50': 0.16, 'G#_75': 0.44, 'G#_90': 0.96, 'A_10

/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_46094/1674305146.py:7: FutureWarning: Pass y=[-0.01960027 -0.03623664 -0.03065557 ...  0.00791283  0.01098923
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectral_centroids = librosa.feature.spectral_centroid(audio, sr=sr)
/var/folders/x0/9kxl__ld5lg60zg21w2jnc2c0000gn/T/ipykernel_46094/1674305146.py:8: FutureWarning: Pass y=[-0.01960027 -0.03623664 -0.03065557 ...  0.00791283  0.01098923
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rolloff = librosa.feature.spectral_rolloff(audio, sr=sr)


In [116]:
sample_track['id']


'2fG8rNGCNw1Wx7y1LjR07r'

In [101]:
import librosa

In [95]:
pitch_df.head(20)

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B
0,1.000,0.815,0.700,0.477,0.365,0.318,0.433,0.598,0.632,0.543,0.493,0.352
1,1.000,0.887,0.549,0.612,0.355,0.191,0.112,0.120,0.079,0.036,0.012,0.183
2,0.090,0.079,0.084,1.000,0.158,0.108,0.147,0.133,0.094,0.094,0.100,0.063
3,0.173,0.230,0.058,1.000,0.085,0.152,0.904,0.175,0.592,0.189,0.164,0.083
4,0.088,0.258,0.187,0.288,0.230,0.991,0.777,0.132,0.250,1.000,0.236,0.080
5,0.046,0.135,0.046,1.000,0.065,0.064,0.070,0.014,0.020,0.096,0.226,0.028
6,0.116,0.419,0.166,0.632,0.115,0.228,0.159,0.144,1.000,0.226,0.196,0.098
7,1.000,0.191,0.109,0.669,0.078,0.145,0.397,0.159,0.698,0.117,0.096,0.102
8,0.120,0.139,0.025,0.021,0.019,0.055,0.135,0.017,0.071,0.030,1.000,0.056
9,0.128,0.320,0.031,0.033,0.018,0.050,0.087,0.014,0.056,0.063,1.000,0.094


In [60]:
print(sample_trackid)
sample_audio_analysis = sp.audio_analysis(sample_trackid)
sample_track = sp.track(sample_trackid)

2fG8rNGCNw1Wx7y1LjR07r


In [61]:
# audio analysis from spotify 

print(sample_audio_analysis.keys())


dict_keys(['meta', 'track', 'bars', 'beats', 'sections', 'segments', 'tatums'])


In [86]:
sample_audio_analysis['segments']


[{'start': 0.0,
  'duration': 0.57297,
  'confidence': 0.0,
  'loudness_start': -60.0,
  'loudness_max_time': 0.52522,
  'loudness_max': -59.319,
  'loudness_end': 0.0,
  'pitches': [1.0,
   0.815,
   0.7,
   0.477,
   0.365,
   0.318,
   0.433,
   0.598,
   0.632,
   0.543,
   0.493,
   0.352],
  'timbre': [0.037,
   170.428,
   8.046,
   -29.479,
   56.211,
   -50.316,
   16.071,
   4.77,
   -26.948,
   1.427,
   -11.128,
   -7.449]},
 {'start': 0.57297,
  'duration': 0.22254,
  'confidence': 0.1,
  'loudness_start': -59.368,
  'loudness_max_time': 0.12642,
  'loudness_max': -57.234,
  'loudness_end': 0.0,
  'pitches': [1.0,
   0.887,
   0.549,
   0.612,
   0.355,
   0.191,
   0.112,
   0.12,
   0.079,
   0.036,
   0.012,
   0.183],
  'timbre': [2.132,
   126.93,
   -62.218,
   -57.05,
   12.051,
   -50.852,
   31.344,
   5.344,
   -26.815,
   6.582,
   -13.159,
   -0.924]},
 {'start': 0.79551,
  'duration': 0.59129,
  'confidence': 1.0,
  'loudness_start': -57.973,
  'loudness_max_t

In [28]:
sample_track.keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [33]:
sample_track['preview_url']

'https://p.scdn.co/mp3-preview/6bbb960159eb9f2a0df0be76b6cd229a0fbfd50b?cid=4f8203f26b84485fb78b0c6a809cba15'

In [44]:
# sample_album = sp.album(sample_track["album"]["external_urls"]["spotify"])
# print("album genres:", sample_album["genres"])

# sample_artist = sp.artist(sample_track["artists"][0]["external_urls"]["spotify"])
# print("artist genres:", sample_artist["genres"])

genres = get_genre_from_track(sp, sample_trackid)
print(genres)

artist genres: ['classical', 'french romanticism', 'late romantic era']
['classical']


In [55]:
audio_file = wget.download(sample_track['preview_url'])
audio_file

100% [............................................................................] 362861 / 362861

'575bad59af93b0f5a10682c333b2e56d2749b9d8'

In [18]:
list_of_genres.head()

,1,2
0,,
1,,pop
2,,dance pop
3,,rap
4,,rock
5,,hip hop


In [5]:
list_of_genres= !wget -O https://everynoise.com/everynoise1d.cgi?scope=all

In [6]:
list_of_genres

['zsh:1: no matches found: https://everynoise.com/everynoise1d.cgi?scope=all']